In [1]:
#安装相关依赖库 如果是windows系统，cmd命令框中输入pip安装，参考上述环境配置
#!pip install sklearn
#!pip install pandas
#---------------------------------------------------
#导入库
#----------------数据探索----------------
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [2]:
#数据预处理
data1=pd.read_csv('../data/比赛训练集.csv',encoding='gbk')
data2=pd.read_csv('../data/比赛测试集.csv',encoding='gbk')

In [3]:
#label标记为-1
data2['患有糖尿病标识']=-1

In [4]:
#训练集和测试机合并
data=pd.concat([data1,data2],axis=0,ignore_index=True)
data

,编号,性别,出生年份,体重指数,糖尿病家族史,舒张压,口服耐糖量测试,胰岛素释放实验,肱三头肌皮褶厚度,患有糖尿病标识
0,1,0,1996,30.1,无记录,106.0,3.818,7.89,0.0,0
1,2,0,1988,27.5,无记录,84.0,-1.000,0.00,14.7,0
2,3,1,1988,36.5,无记录,85.0,7.131,0.00,40.1,1
3,4,1,1992,29.5,无记录,91.0,7.041,0.00,0.0,0
4,5,0,1998,42.0,叔叔或者姑姑有一方患有糖尿病,NaN,7.134,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
6065,996,1,1990,50.1,无记录,87.0,5.125,0.00,0.0,-1
6066,997,0,1992,56.3,无记录,87.0,7.695,0.00,0.0,-1
6067,998,1,1992,23.8,无记录,85.0,3.194,7.50,0.0,-1
6068,999,0,2000,53.1,无记录,95.0,8.226,7.55,0.0,-1


In [5]:
#将舒张压特征中的缺失值填充为-1
data['舒张压']=data['舒张压'].fillna(-1)

# 特征工程

In [6]:
"""
将出生年份换算成年龄
"""
data['年龄']=2022-data['出生年份']  #换成年龄

In [7]:
"""
人体的成人体重指数正常值是在18.5-24之间
低于18.5是体重指数过轻
在24-27之间是体重超重
27以上考虑是肥胖
高于32了就是非常的肥胖。
"""
def BMI(a):
    if a<18.5:
        return 0
    elif 18.5<=a<=24:
        return 1
    elif 24<a<=27:
        return 2
    elif 27<a<=32:
        return 3
    else:
        return 4

data['BMI']=data['体重指数'].apply(BMI)

In [8]:
#糖尿病家族史
"""
无记录
叔叔或者姑姑有一方患有糖尿病/叔叔或姑姑有一方患有糖尿病
父母有一方患有糖尿病
"""
def FHOD(a):
    if a=='无记录':
        return 0
    elif a=='叔叔或者姑姑有一方患有糖尿病' or a=='叔叔或姑姑有一方患有糖尿病':
        return 1
    else:
        return 2

data['糖尿病家族史']=data['糖尿病家族史'].apply(FHOD)

In [9]:
"""
舒张压范围为60-90
"""
def DBP(a):
    if a<60:
        return 0
    elif 60<=a<=90:
        return 1
    elif a>90:
        return 2
    else:
        return a
data['DBP']=data['舒张压'].apply(DBP)

In [10]:
#------------------------------------
#将处理好的特征工程分为训练集和测试集，其中训练集是用来训练模型，测试集用来评估模型准确度
#其中编号和患者是否得糖尿病没有任何联系，属于无关特征予以删除
train=data[data['患有糖尿病标识'] !=-1]
test=data[data['患有糖尿病标识'] ==-1]
train_label=train['患有糖尿病标识']
train=train.drop(['编号','患有糖尿病标识','出生年份'],axis=1)
test=test.drop(['编号','患有糖尿病标识','出生年份'],axis=1)

In [11]:
train

,性别,体重指数,糖尿病家族史,舒张压,口服耐糖量测试,胰岛素释放实验,肱三头肌皮褶厚度,年龄,BMI,DBP
0,0,30.1,0,106.0,3.818,7.89,0.0,26,3,2
1,0,27.5,0,84.0,-1.000,0.00,14.7,34,3,1
2,1,36.5,0,85.0,7.131,0.00,40.1,34,4,1
3,1,29.5,0,91.0,7.041,0.00,0.0,30,3,2
4,0,42.0,1,-1.0,7.134,0.00,0.0,24,4,0
...,...,...,...,...,...,...,...,...,...,...
5065,1,36.4,2,95.0,3.102,0.00,0.0,30,4,2
5066,1,37.1,1,94.0,6.207,6.77,0.0,31,4,2
5067,0,25.4,0,90.0,7.343,0.00,0.0,47,2,1
5068,0,28.5,2,101.0,6.268,8.99,0.0,50,3,2


In [12]:
#----------------模型训练----------------
model = DecisionTreeClassifier()
model.fit(train, train_label) 
y_pre=model.predict(test)
y_pre

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,

In [13]:
#----------------结果输出----------------
result=pd.read_csv('../data/提交示例.csv')
result['label']=y_pre
result.to_csv('result-de.csv',index=False)